In [1]:
#

In [2]:
#

In [3]:
! pip install jiwer

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
train_df = pd.read_csv("/kaggle/input/ipa-dataverse-added/output_dataverse.csv")
train_df = train_df[["Contents", "IPA"]]

train_df, val_df = train_test_split(train_df, test_size=0.0001, shuffle=True, random_state=3000)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [6]:
from datasets import Dataset

ds_train = Dataset.from_pandas(train_df)
ds_eval = Dataset.from_pandas(val_df)

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model_id = "google/byt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer)

2024-02-23 04:10:12.499102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 04:10:12.499203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 04:10:12.633635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
def prepare_dataset(sample):
    output = tokenizer(sample["Contents"])
    output["labels"] = tokenizer(sample["IPA"])['input_ids']
    output["length"] = len(output["labels"])
    return output


ds_train = ds_train.map(prepare_dataset, remove_columns=ds_train.column_names)
ds_eval = ds_eval.map(prepare_dataset, remove_columns=ds_eval.column_names)

  0%|          | 0/52304 [00:00<?, ?ex/s]

  0%|          | 0/6 [00:00<?, ?ex/s]

In [9]:
import numpy as np
from datasets import load_metric

wer_metric = load_metric("wer")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"wer": result}

In [10]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    group_by_length=True,
    length_column_name="length",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    metric_for_best_model="wer",
    greater_is_better=False,
    load_best_model_at_end=True,
    num_train_epochs=1,
    save_steps=13000,
    eval_steps=13000,
    logging_steps=13000,
    learning_rate=1e-4,
    weight_decay=5e-3,
    warmup_steps=500,
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=512,
    push_to_hub=False,
    report_to="none",
)

In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Wer
13000,0.121900,0.015848,0.000000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=13076, training_loss=0.12129524470506074, metrics={'train_runtime': 6332.9937, 'train_samples_per_second': 8.259, 'train_steps_per_second': 2.065, 'total_flos': 2.465348209355981e+16, 'train_loss': 0.12129524470506074, 'epoch': 1.0})

In [12]:
trainer.save_model(model_id)

In [13]:
import pandas as pd
from transformers import pipeline

In [14]:
test_df = pd.read_csv("/kaggle/input/regipa/test_regipa.csv")

In [15]:
pipe = pipeline("text2text-generation", model=model_id, device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
# preprocess-1: sort by length
index = test_df["Contents"].str.len().sort_values(ascending=False).index
test_df = test_df.reindex(index)

In [17]:
# preprocess-2: removing english alphabets
alpha_pat = "[a-zA-Z]"
test_df["Contents"] = test_df["Contents"].str.replace(alpha_pat, "", regex=True)

In [18]:
# preprocess-3: adding period at the end of those texts not having period or '।'
indices_to_add_period = test_df[~test_df['Contents'].str.endswith('।')].index
test_df.loc[indices_to_add_period, 'Contents'] += '।'

In [19]:
%%time
texts = test_df["Contents"].tolist()
ipas = pipe(texts, max_length=512, batch_size=64)
ipas = [ipa["generated_text"] for ipa in ipas]

CPU times: user 6min 49s, sys: 703 ms, total: 6min 49s
Wall time: 6min 49s


In [20]:
# postprocess-1
test_df["IPA"] = ipas
test_df = test_df.sort_index()

In [21]:
# postprocess-2: removing period from the end of those ipas in which texts period were added in preprocess-3
test_df.loc[indices_to_add_period, 'IPA'] = test_df.loc[indices_to_add_period, 'IPA'].str.rstrip('।')
test_df.head()

,Index,District,Contents,IPA
0,0,Rangpur,এলা সবায় সবার হাতোত <> অসহায় মানুষ আচে?।,elɐ ʃɔbɐe̯ ʃɔbɐɾ hɐt̪ot̪ <> ɔʃohɐe̯ mɐnuʃ ɐce?
1,1,Rangpur,কেউ কারো ইয়া নাই।,ke͡u̯ kɐɾo ɪʲɐ nɐ͡ɪ̯।
2,2,Rangpur,"এলা ওই যে, কাইলকা ব্যাটায় ইপতারি আনচে, খাইচোং,...","elɐ o͡ɪ̯ ɟe, kɐ͡ɪ̯lkɐ bɛtɐe̯ ɪpot̪ɐɾɪ ɐnce, kʰ..."
3,3,Rangpur,আর মুই আগোত কী করচিনু?।,ɐɾ mu͡ɪ̯ ɐgot̪ kɪ koɾcɪnu?
4,4,Rangpur,"<> আগের কতা বাদ দেও, এলা নাই।","<> ɐgeɾ kɔt̪ɐ bɐd̪ d̪e͡o̯, elɐ nɐ͡ɪ̯।"


In [22]:
test_df.to_csv("submission2.csv", columns=["IPA"])

In [23]:
df = pd.read_csv('/kaggle/working/submission2.csv')


# Check if there's any unnamed column
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]

if unnamed_columns:
    # Rename the first unnamed column to a desired name
    new_name = 'id'  # Specify your desired name here
    old_name = unnamed_columns[0]
    df.rename(columns={old_name: new_name}, inplace=True)


# Display the DataFrame after renaming the columns
print("\nDataFrame after renaming columns:")
df.rename(columns={'IPA': 'string'}, inplace=True)
print(df)



DataFrame after renaming columns:
        id                                             string
0        0    elɐ ʃɔbɐe̯ ʃɔbɐɾ  hɐt̪ot̪ <> ɔʃohɐe̯ mɐnuʃ ɐce?
1        1                              ke͡u̯ kɐɾo ɪʲɐ nɐ͡ɪ̯।
2        2  elɐ o͡ɪ̯ ɟe, kɐ͡ɪ̯lkɐ bɛtɐe̯ ɪpot̪ɐɾɪ ɐnce, kʰ...
3        3                         ɐɾ mu͡ɪ̯ ɐgot̪ kɪ koɾcɪnu?
4        4              <> ɐgeɾ kɔt̪ɐ bɐd̪ d̪e͡o̯, elɐ nɐ͡ɪ̯।
...    ...                                                ...
8936  8936           pɔɾe pʰɾedɪ t̪ɔkʰon puɾɐ bʱɐ͡ɪ̯ŋgɐ pɔɾe।
8937  8937  ɐɾ pʰɾedɪɾ ʃɐt̪ʰe ɐɾo kɪcʰu mɐnuʃeɾ poɾɪcɔe̯ h...
8938  8938  ɟe ɟɐɾɐ ɐʃole o͡ɪ̯ ɟɐɾɐ elbɐm ɾɪlɪɟ d̪ee̯ o͡ɪ̯...
8939  8939  ɟɐɾɐ ɐʃole pʰɾedɪɾ e͡ɪ̯ ɟɔnopɾɪʲot̪ɐdɐ ɐʃole t...
8940  8940                       t̪ɐɾɐ cɐ͡ɪ̯cʰɪlo pʰɾedɪ ɟɐnɪ

[8941 rows x 2 columns]


In [24]:
df.isnull().sum()

id        0
string    0
dtype: int64

In [25]:
null_rows = df[df.isnull().any(axis=1)]
print("Rows with null values:")
print(null_rows)

Rows with null values:
Empty DataFrame
Columns: [id, string]
Index: []


In [26]:
df_filled = df.fillna('missing')
print(df_filled)

        id                                             string
0        0    elɐ ʃɔbɐe̯ ʃɔbɐɾ  hɐt̪ot̪ <> ɔʃohɐe̯ mɐnuʃ ɐce?
1        1                              ke͡u̯ kɐɾo ɪʲɐ nɐ͡ɪ̯।
2        2  elɐ o͡ɪ̯ ɟe, kɐ͡ɪ̯lkɐ bɛtɐe̯ ɪpot̪ɐɾɪ ɐnce, kʰ...
3        3                         ɐɾ mu͡ɪ̯ ɐgot̪ kɪ koɾcɪnu?
4        4              <> ɐgeɾ kɔt̪ɐ bɐd̪ d̪e͡o̯, elɐ nɐ͡ɪ̯।
...    ...                                                ...
8936  8936           pɔɾe pʰɾedɪ t̪ɔkʰon puɾɐ bʱɐ͡ɪ̯ŋgɐ pɔɾe।
8937  8937  ɐɾ pʰɾedɪɾ ʃɐt̪ʰe ɐɾo kɪcʰu mɐnuʃeɾ poɾɪcɔe̯ h...
8938  8938  ɟe ɟɐɾɐ ɐʃole o͡ɪ̯ ɟɐɾɐ elbɐm ɾɪlɪɟ d̪ee̯ o͡ɪ̯...
8939  8939  ɟɐɾɐ ɐʃole pʰɾedɪɾ e͡ɪ̯ ɟɔnopɾɪʲot̪ɐdɐ ɐʃole t...
8940  8940                       t̪ɐɾɐ cɐ͡ɪ̯cʰɪlo pʰɾedɪ ɟɐnɪ

[8941 rows x 2 columns]


In [27]:
df_filled.to_csv('submission.csv', index=False)